In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
batch_size = 20

In [3]:
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D

Using TensorFlow backend.


In [4]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [8]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input


In [10]:
train_data = np.load('../data/train_data.npz')
X_train, y_train = preprocess_input(train_data['images']), train_data['labels']

In [18]:
_input = Input(shape=(224, 224, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor = _input)

for idx, layer in vgg16.layers_by_depth.items():
    if idx in [1, 2, 3]:
        layer[0].trainable = True
    else:
        layer[0].trainable = False
        
_layer2 = Flatten()(vgg16.output)
_layer3 = Dense(200, activation='relu')(_layer2)
_output = Dense(1, activation='sigmoid')(_layer3)

model = Model(inputs=[_input], outputs=[_output])
optimizer = Adam(lr=0.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
from keras.callbacks import ReduceLROnPlateau

lrdecay = ReduceLROnPlateau(patience=5)

results = model.fit(X_train, y_train, epochs=50,
                    batch_size=128, validation_split=0.2,
                    callbacks=[lrdecay]
                   )

Train on 1836 samples, validate on 459 samples
Epoch 1/50


In [ ]:
from matplotlib import pyplot as plt
plt.subplot(2, 1, 1)
plt.plot(results.epoch, np.log(results.history['loss']), label='loss')
plt.plot(results.epoch, np.log(results.history['val_loss']), label='val_loss')
plt.ylabel('loss')
plt.legend()
plt.subplot(2, 1, 2)
plt.plot(results.epoch, np.log(results.history['acc']), label='acc')
plt.plot(results.epoch, np.log(results.history['val_acc']), label='val_acc')
plt.ylabel('acc')
plt.xlabel('epochs')
plt.legend()
plt.show()